<a href="https://colab.research.google.com/github/gtoubian/cce/blob/main/Project_3_Gabriel_Toubian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Project 3: NHL Player Salaries**
**By: Gabriel Toubian**

This project will involve the yearly salaries for NHL players and the different factors that influence them. We can make inferences that determine what makes a player more "valuable" to an NHL team (earn a higher salary).

**Importing the dataset and building a pandas dataframe out of it**

In [ ]:
from google.colab import files
data = files.upload()

Saving nhl.csv to nhl (2).csv


In [ ]:
import pandas as pd

import io
nhl = pd.read_csv('nhl.csv', encoding = "ISO-8859-1")

In [ ]:
nhl = nhl.dropna(how='any')

In [ ]:
nhl.tail()

,Salary,Born,City,Pr/St,Cntry,Nat,Ht,Wt,DftYr,DftRd,Ovrl,Hand,Last Name,First Name,Position,Team,GP,G,A,A1,A2,PTS,+/-,E+/-,PIM,Shifts,TOI,TOIX,TOI/GP,TOI/GP.1,TOI%,IPP%,SH%,SV%,PDO,F/60,A/60,Pct%,Diff,Diff/60,...,Min,Maj,Match,Misc,Game,CF,CA,FF,FA,SF,SA,xGF,xGA,SCF,SCA,GF,GA,RBF,RBA,RSF,RSA,DSF,DSA,FOW,FOL,HF,HA,GVA,TKA,PENT,PEND,OPS,DPS,PS,OTOI,Grit,DAP,Pace,GS,GS/G
606,680000,93-04-16,Burnsville,MN,USA,USA,77,210,2011.0,4.0,101.0,L,Labate,Joseph,C,VAN,13,0,0,0,0,0,-2,-1.3,21,147,5276,88.0,6.77,6.77,13.1,0.0,6.3,0.909,972.0,1.36,2.73,33.3,-2,-1.36,...,3,3,0,0,0,68.0,89.0,47.0,66.0,32.0,44.0,2.3,3.2,8.0,7.0,2.0,4.0,0.0,5.0,6.0,5.0,6,10,35.0,35.0,54.0,42.0,13.0,11.0,9.0,4.0,-0.2,0.0,-0.2,581.67,49,8.7,107.1,-1.2,-0.09
607,600000,92-09-26,Denver,CO,USA,USA,73,201,2011.0,3.0,82.0,R,Shore,Nick,C,L.A,70,6,11,6,5,17,-2,4.2,20,1258,53712,894.6,12.78,12.78,22.8,65.4,6.6,0.920,985.0,1.74,2.01,46.4,-4,-0.27,...,10,0,0,0,0,780.0,764.0,569.0,569.0,395.0,373.0,32.9,33.6,106.0,106.0,26.0,30.0,33.0,20.0,38.0,38.0,71,58,383.0,382.0,439.0,429.0,113.0,48.0,55.0,47.0,0.0,1.3,1.3,3023.36,100,6.9,103.6,23.2,0.33
608,9000000,85-01-21,Madison,WI,USA,USA,74,206,2003.0,1.0,7.0,L,Suter,Ryan,D,MIN,82,9,31,10,21,40,34,19.7,36,2319,132469,2190.7,26.93,26.86,44.7,34.5,10.2,0.927,1029.0,3.18,2.14,59.8,38,1.04,...,18,0,0,0,0,2056.0,2009.0,1546.0,1510.0,1137.0,1073.0,107.8,82.6,361.0,244.0,116.0,78.0,110.0,50.0,98.0,52.0,208,102,1257.0,1122.0,513.0,583.0,198.0,247.0,95.0,112.0,2.7,6.6,9.3,2715.54,213,6.3,111.3,44.1,0.54
609,4250000,87-03-30,Montreal,QC,CAN,CAN,73,205,2005.0,2.0,35.0,L,Vlasic,Marc-Edouard,D,S.J,75,6,22,14,8,28,4,0.6,35,1982,95555,1573.0,21.07,21.09,35.5,43.8,9.2,0.908,1000.0,2.44,2.52,49.2,-2,-0.08,...,10,1,0,0,1,1334.0,1519.0,980.0,1038.0,693.0,714.0,60.1,65.0,196.0,215.0,64.0,66.0,42.0,42.0,60.0,71.0,102,113,739.0,833.0,426.0,513.0,311.0,189.0,67.0,75.0,1.7,4.1,5.8,2855.83,200,4.1,108.8,32.5,0.43
611,925000,93-05-12,St. Albert,AB,CAN,CAN,78,226,2012.0,3.0,86.0,R,Parayko,Colton,D,STL,81,4,31,17,14,35,7,1.7,32,2365,103018,1712.4,21.20,21.18,35.4,46.1,9.2,0.917,1009.0,2.66,2.24,54.3,12,0.42,...,16,0,0,0,0,1507.0,1462.0,1127.0,1081.0,826.0,771.0,65.8,66.0,208.0,222.0,76.0,64.0,45.0,43.0,62.0,54.0,107,97,813.0,815.0,578.0,642.0,150.0,174.0,100.0,100.0,2.2,4.5,6.7,3119.42,229,7.4,104.0,45.8,0.56


We can see this dataset includes the salaries and stats for 612 players.

# Hypothesis testing

Whether a player is left or right handed can have on impact on the type of position they can play optimally. Therefore, do players of a certain handedness come at a premium? Let's start by seeing if whether a player is **left or right handed** has an impact on their salary. We can run a one-way ANOVA test to see if this factor is statistically significant.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import scipy.stats as sc

In [ ]:
nhlhand = nhl[["Hand", "Salary"]]
Left = np.array(nhlhand[nhlhand["Hand"]=="L"]["Salary"])
Right = np.array(nhlhand[nhlhand["Hand"]=="R"]["Salary"])

sc.f_oneway(Left, Right)

F_onewayResult(statistic=0.03681398777778426, pvalue=0.8479533459419761)

With a p-value of well over 0.05, we cannot reject the null hypothesis, which means that a player's handedness doesn't necessarily play a sigificant part in determining their salary.

How about a player's **Nationality**? Do certain countries have a tendency of producing more "valuable" players through their programs? Let's take 2 countries that have a tendency to finish relatively high in country-based competitions: **Canada and USA**

In [ ]:
nhlnat = nhl[["Nat", "Salary"]]

In [ ]:
nhlnat = nhlnat[(nhlnat.Nat == 'CAN') | (nhlnat.Nat == 'USA')]

In [ ]:
Canadian = np.array(nhlnat[nhlnat["Nat"]=="CAN"]["Salary"])
American = np.array(nhlnat[nhlnat["Nat"]=="USA"]["Salary"])

sc.f_oneway(Canadian, American)

F_onewayResult(statistic=0.019945699195699655, pvalue=0.8877687260535069)

With a p-value of well over 0.05, we cannot reject the null hypothesis, which means that a player's nationality (in this case, whether they are Canadian or American) doesn't necessarily play a sigificant part in determining their salary.

# Multiple Regression

Let's build a regressional model to see which numerical features play a significant part in determining a player's salary. Not adding a constant yields a higher adjusted R-squared, so we will not add one.

In [ ]:
nhl2=nhl.copy()

In [ ]:
del nhl2['Salary']
del nhl2['Born']
del nhl2['City']
del nhl2['Pr/St']
del nhl2['Cntry']
del nhl2['Nat']
del nhl2['DftYr']
del nhl2['Hand']
del nhl2['Last Name']
del nhl2['First Name']
del nhl2['Position']
del nhl2['Team']

In [ ]:
X = nhl2
y = nhl['Salary']
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Salary   R-squared (uncentered):                   0.904
Model:                            OLS   Adj. R-squared (uncentered):              0.848
Method:                 Least Squares   F-statistic:                              16.01
Date:                Fri, 23 Apr 2021   Prob (F-statistic):                    7.72e-69
Time:                        19:23:22   Log-Likelihood:                         -5493.7
No. Observations:                 359   AIC:                                  1.125e+04
Df Residuals:                     226   BIC:                                  1.177e+04
Df Model:                         133                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Ht         -1.049e+05   6.03e+04     -1.740      0.083   -2.24e+05    1.39e+04
Wt          2.711e+04   9264.395      2.926      0.004    8856.513    4.54e+04
DftRd      -1.762e+05   3.08e+05     -0.572      0.568   -7.83e+05    4.31e+05
Ovrl        4083.7337   9921.463      0.412      0.681   -1.55e+04    2.36e+04
GP         -7.935e+04   9.09e+04     -0.873      0.384   -2.58e+05    9.98e+04
G          -2.674e+06   3.09e+06     -0.865      0.388   -8.76e+06    3.41e+06
A          -9.474e+06   2.78e+06     -3.412      0.001   -1.49e+07      -4e+06
A1          5.755e+06   2.17e+06      2.657      0.008    1.49e+06       1e+07
A2          5.715e+06   2.16e+06      2.645      0.009    1.46e+06    9.97e+06
PTS         3.899e+06   1.92e+06      2.032      0.043    1.19e+05    7.68e+06
+/-        -2902.2398    3.1e+04     -0.094      0.926    -6.4e+04    5.82e+04
E+/-        4.434e+04   4.79e+04      0.925      0.356   -5.01e+04    1.39e+05
PIM         1.181e+04    9.6e+04      0.123      0.902   -1.77e+05    2.01e+05
Shifts     -3462.9948   1476.993     -2.345      0.020   -6373.433    -552.557
TOI         -417.0925    556.409     -0.750      0.454   -1513.506     679.321
TOIX        3.197e+04   3.33e+04      0.960      0.338   -3.37e+04    9.76e+04
TOI/GP      1.198e+06   2.34e+06      0.513      0.609    -3.4e+06     5.8e+06
TOI/GP.1    -8.99e+05   2.34e+06     -0.384      0.702   -5.52e+06    3.72e+06
TOI%       -1.628e+05   1.69e+05     -0.966      0.335   -4.95e+05    1.69e+05
IPP%       -4034.9291   5179.287     -0.779      0.437   -1.42e+04    6170.940
SH%         7.482e+05    1.9e+06      0.394      0.694   -2.99e+06    4.49e+06
SV%         7.055e+07   1.89e+08      0.374      0.709   -3.01e+08    4.42e+08
PDO        -6.772e+04   1.89e+05     -0.359      0.720    -4.4e+05    3.04e+05
F/60       -2.205e+07    1.8e+07     -1.223      0.223   -5.76e+07    1.35e+07
A/60        2.186e+07    1.8e+07      1.212      0.227   -1.37e+07    5.74e+07
Pct%       -6336.6469   9267.814     -0.684      0.495   -2.46e+04    1.19e+04
Diff       -1.647e+04   1.78e+04     -0.926      0.355   -5.15e+04    1.86e+04
Diff/60     2.184e+07    1.8e+07      1.210      0.227   -1.37e+07    5.74e+07
iCF        -1.834e+05    7.5e+04     -2.444      0.015   -3.31e+05   -3.55e+04
iCF.1       1.866e+05   7.65e+04      2.440      0.015    3.59e+04    3.37e+05
iFF         7.268e+05   3.29e+05      2.211      0.028    7.92e+04    1.37e+06
iSF        -1.037e+06   4.19e+05     -2.476      0.014   -1.86e+06   -2.12e+05
iSF.1       2.768e+05   8.36e+05      0.331      0.741   -1.37e+06    1.92e+06
iSF.2      -1.708e+06   1.97e+06     -0.866      0.388    -5.6e+06    2.18e+06
ixG        -1255.2677   2.74e+05     -0.005      0.996    -5.

We can now export this data to an Excel sheet then re-import it and create a Pandas Dataframe so we can proceed with Feature Selection.

Selected features will have a p-value of under 0.05

In [ ]:
data = files.upload()

Saving regress.csv to regress (2).csv


In [ ]:
features = pd.read_csv('regress.csv', encoding = "ISO-8859-1")

In [ ]:
features.head()

,Stat,coef,std err,t,P>|t|,[0.025,0.975]
0,Ht,-105000.0000,60300.000,-1.740,0.083,-224000.000,13900.0
1,Wt,27100.0000,9264.395,2.926,0.004,8856.513,45400.0
2,DftRd,-176000.0000,308000.000,-0.572,0.568,-783000.000,431000.0
3,Ovrl,4083.7337,9921.463,0.412,0.681,-15500.000,23600.0
4,GP,-79400.0000,90900.000,-0.873,0.384,-258000.000,99800.0


In [ ]:
features = features.rename({'P>|t|':'pval'}, axis=1)

In [ ]:
selected = features[(features.pval < 0.05)]

In [ ]:
selected

,Stat,coef,std err,t,pval,[0.025,0.975]
1,Wt,2.710000e+04,9264.395,2.926,0.004,8.856513e+03,4.540000e+04
6,A,-9.470000e+06,2780000.000,-3.412,0.001,-1.490000e+07,-4.000000e+06
7,A1,5.760000e+06,2170000.000,2.657,0.008,1.490000e+06,1.000000e+07
8,A2,5.720000e+06,2160000.000,2.645,0.009,1.460000e+06,9.970000e+06
9,PTS,3.900000e+06,1920000.000,2.032,0.043,1.190000e+05,7.680000e+06
13,Shifts,-3.462995e+03,1476.993,-2.345,0.020,-6.373433e+03,-5.525570e+02
28,iCF,-1.830000e+05,75000.000,-2.444,0.015,-3.310000e+05,-3.550000e+04
29,iCF.1,1.870000e+05,76500.000,2.440,0.015,3.590000e+04,3.370000e+05
30,iFF,7.270000e+05,329000.000,2.211,0.028,7.920000e+04,1.370000e+06
31,iSF,-1.040000e+06,419000.000,-2.476,0.014,-1.860000e+06,-2.120000e+05


In [ ]:
selected = selected.drop(6)

We can drop feature "A" as it is redundant (it is overall assists i.e. the sum of features "A1 and A2"



In [ ]:
selected = selected.drop(13)

We can drop feature "Shifts" as its significance is too ambiguous.

In [ ]:
selected = selected.drop(28)

We can drop feature "iCF" as it is redundant 

In [ ]:
selected = selected.drop(47)

We can drop feature "iGVA" as it is redundant

In [ ]:
selected

,Stat,coef,std err,t,pval,[0.025,0.975]
1,Wt,2.710000e+04,9264.395,2.926,0.004,8856.513,4.540000e+04
7,A1,5.760000e+06,2170000.000,2.657,0.008,1490000.000,1.000000e+07
8,A2,5.720000e+06,2160000.000,2.645,0.009,1460000.000,9.970000e+06
9,PTS,3.900000e+06,1920000.000,2.032,0.043,119000.000,7.680000e+06
29,iCF.1,1.870000e+05,76500.000,2.440,0.015,35900.000,3.370000e+05
30,iFF,7.270000e+05,329000.000,2.211,0.028,79200.000,1.370000e+06
31,iSF,-1.040000e+06,419000.000,-2.476,0.014,-1860000.000,-2.120000e+05
46,iMiss,-5.940000e+05,263000.000,-2.256,0.025,-1110000.000,-7.520000e+04
50,iGVA.1,1.350000e+06,368000.000,3.674,0.000,627000.000,2.080000e+06
68,FOW.Down,6.050000e+04,28500.000,2.126,0.035,4436.329,1.170000e+05


After removing redundant features, the features that seem to be the most significant (p-value below 0.05) are:

Wt - Weight

A1 - First assists, primary assists

A2 - Second assists, secondary assists

PTS - Points. Goals plus all assists

iCF.1 - Shot attempts taken by this individual

iFF - Unblocked shot attempts taken by this individual

iSF - Shots on goal taken by this individual

iMiss - Individual shots taken that missed the net.

iGVA.1 - Giveaways by this individual

FOW.Down - Faceoffs won when the team was trailing

FOL.Down - Faceoffs lost when the team was trailing

FOL.Close - Faceoffs lost when the score was close

Min - Minor penalties taken

Maj - Major penalties taken

Match - Match penalties

HF - The team's hits thrown while this player was on the ice


**Some of these features are more surprising than others yet make sense when you think about them in context.**

Now that we have our selected features, we can start modeling


# Train / Test Split

Let's create a sub-dataset by filtering our existing dataset to only the selected features:

In [ ]:
subfeatures = nhl[["Wt", "A1", "A2", "PTS", "iCF.1", "iFF", "iSF", "iMiss", "iGVA.1", "FOW.Down", "FOL.Down", "FOL.Close", "Min", "Maj", "Match", "HF"]].copy()

In [ ]:
subsalary = nhl[["Salary"]].copy()

We can split this data into Training at Test sets (70/30)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = subfeatures
y = subsalary
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

# Linear Regression Model

How that we have our training and test sets, we can create a linear regression model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lrmodel = LinearRegression().fit(X_train, y_train)
print(lrmodel.intercept_)
print(lrmodel.coef_)

[-6061355.47408668]
[[ 3.33578402e+04  2.52804754e+04  3.98082523e+04  3.76482124e+04
  -4.82076661e+03  1.25589417e+05 -1.08906389e+05 -1.36607305e+05
   1.87695304e+04  2.98995660e+04 -1.31126699e+04 -1.07124572e+04
   4.58854612e+03 -4.74626899e+04 -1.68810006e+06 -5.52276533e+02]]


Wt - Weight

A1 - First assists, primary assists

A2 - Second assists, secondary assists

PTS - Points. Goals plus all assists

iCF.1 - Shot attempts taken by this individual

iFF - Unblocked shot attempts taken by this individual

iSF - Shots on goal taken by this individual

iMiss - Individual shots taken that missed the net.

iGVA.1 - Giveaways by this individual

FOW.Down - Faceoffs won when the team was trailing

FOL.Down - Faceoffs lost when the team was trailing

FOL.Close - Faceoffs lost when the score was close

Min - Minor penalties taken

Maj - Major penalties taken

Match - Match penalties

HF - The team's hits thrown while this player was on the ice

Let's see the R2 Value for our model

**When we look at the coefficients of the various features, we can draw conclusions that make sense in context.**

In [ ]:
print(lrmodel.score(X_train, y_train))
print(lrmodel.score(X_test, y_test))

0.528444373676378
0.5308825904523109


In [ ]:
est = sm.OLS(y_train, X_train).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Salary   R-squared (uncentered):                   0.761
Model:                            OLS   Adj. R-squared (uncentered):              0.745
Method:                 Least Squares   F-statistic:                              46.80
Date:                Fri, 23 Apr 2021   Prob (F-statistic):                    1.01e-63
Time:                        18:47:13   Log-Likelihood:                         -3948.3
No. Observations:                 251   AIC:                                      7929.
Df Residuals:                     235   BIC:                                      7985.
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Wt          3146.4449   1083.560      2.904      0.004    1011.713    5281.177
A1          4.675e+04   5.16e+04      0.906      0.366   -5.49e+04    1.48e+05
A2          5.267e+04   4.88e+04      1.080      0.281   -4.34e+04    1.49e+05
PTS         1.969e+04   3.18e+04      0.619      0.536   -4.29e+04    8.23e+04
iCF.1      -5789.1211   8647.225     -0.669      0.504   -2.28e+04    1.12e+04
iFF         5.035e+04   2.38e+05      0.212      0.832   -4.18e+05    5.19e+05
iSF        -3.256e+04   2.38e+05     -0.137      0.891   -5.02e+05    4.37e+05
iMiss       -5.44e+04   2.37e+05     -0.230      0.818    -5.2e+05    4.12e+05
iGVA.1      1.652e+04   1.14e+04      1.455      0.147   -5851.546    3.89e+04
FOW.Down    3.153e+04   8160.438      3.864      0.000    1.55e+04    4.76e+04
FOL.Down   -1.622e+04   1.43e+04     -1.131      0.259   -4.45e+04     1.2e+04
FOL.Close  -9990.3335   7791.390     -1.282      0.201   -2.53e+04    5359.562
Min         7954.5469   2.22e+04      0.358      0.721   -3.59e+04    5.18e+04
Maj           47.8764   6.52e+04      0.001      0.999   -1.28e+05    1.28e+05
Match       -1.82e+06   1.77e+06     -1.028      0.305   -5.31e+06    1.67e+06
HF          -561.4590   1064.180     -0.528      0.598   -2658.011    1535.093
==============================================================================
Omnibus:                       58.402   Durbin-Watson:                   2.102
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.130
Skew:                           1.034   Prob(JB):                     1.25e-34
Kurtosis:                       6.263   Cond. No.                     8.62e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# "Rich" or "Not"

For argument's sake, let's say that a player who makes a salary above the average is considered "rich" and one who makes below the average is considered "not". Let's start by seeing what the average salary is

In [ ]:
nhl["Salary"].mean()

2456543.52367688

Now that we have the average salary, we can make the column to determine if the player is above average or not. A "1" designates above average salary and a "0" designates below average.

In [ ]:
subfeatures['rich'] = np.where(subsalary['Salary'] > 2456543, '1', '0')

In [ ]:
subfeatures.head()

,Wt,A1,A2,PTS,iCF.1,iFF,iSF,iMiss,iGVA.1,FOW.Down,FOL.Down,FOL.Close,Min,Maj,Match,HF,rich
0,190,0,0,0,2,2.0,1.0,1,1.0,0,0,0,0,0,0,1.0,0
1,207,6,9,17,287,197.0,143.0,54,74.0,0,0,0,10,0,0,749.0,0
2,218,13,13,45,283,212.0,155.0,57,36.0,21,16,33,12,0,0,340.0,1
3,220,5,0,6,88,55.0,40.0,15,23.0,0,0,0,4,2,0,198.0,1
4,217,4,8,19,166,118.0,95.0,23,21.0,0,3,6,33,9,0,512.0,0


Let's run a Logistic Regression Model to predict if a player is above average salary based on the selected features

In [ ]:
logresfeat = nhl[["Wt", "A1", "A2", "PTS", "iCF.1", "iFF", "iSF", "iMiss", "iGVA.1", "FOW.Down", "FOL.Down", "FOL.Close", "Min", "Maj", "Match", "HF"]].copy()

In [ ]:
a = logresfeat
b = subfeatures['rich']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=0.3, random_state=0)
lr = LogisticRegression().fit(a_train, b_train)
bhat = lr.predict(a_train)

dftrain = pd.DataFrame(a_train, columns = ["Wt", "A1", "A2", "PTS", "iCF.1", "iFF", "iSF", "iMiss", "iGVA.1", "FOW.Down", "FOL.Down", "FOL.Close", "Min", "Maj", "Match", "HF"])
dftrain['Actual'] = b_train
dftrain['Predicted'] = bhat
dftrain.head()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,Wt,A1,A2,PTS,iCF.1,iFF,iSF,iMiss,iGVA.1,FOW.Down,FOL.Down,FOL.Close,Min,Maj,Match,HF,Actual,Predicted
467,215,5,7,25,153,127.0,87.0,40,30.0,4,8,24,13,1,0,375.0,1,0
338,200,1,0,1,18,12.0,9.0,3,1.0,10,9,13,2,0,0,60.0,0,0
302,185,9,5,23,164,134.0,95.0,39,20.0,9,10,17,8,0,0,489.0,0,0
489,193,15,12,40,226,176.0,133.0,43,36.0,155,152,329,15,1,0,427.0,0,0
255,198,0,0,0,8,7.0,3.0,4,3.0,1,0,1,0,0,0,31.0,0,0


In [ ]:
lr.coef_

array([[-0.01421086,  0.02043752, -0.09145005,  0.07461333,  0.01440079,
        -0.0105717 , -0.00094363, -0.02351306,  0.02497234,  0.05300078,
        -0.03285204, -0.01281611, -0.0112159 , -0.04885372, -0.00729781,
         0.00130981]])

In [ ]:
lr.predict_proba(a_test)[0]

array([0.90145121, 0.09854879])

In [ ]:
accuracy_score(b_train, bhat)

0.7928286852589641

In [ ]:
bhat = lr.predict(a_test)
accuracy_score(b_test, bhat)

0.8055555555555556

We have an accuracy score of about 80.5% on our logistic regression model to determine if a player is above or below average salary based on the selected features.

# Tableau - Points Production vs Salary; Value Overview

Tableau workbook gives an overview of players point production versus the salary paid. Gives an idea of players who can be considered a "great bragain" vs those who are "overpaid" when it comes to point production.